In [32]:
import os, zipfile
import re
import pandas as pd
import fastqcparser
from pprint import pprint
from fastqcparser import FastQCParser
from pathlib import Path
from glob import glob
import numpy as np

In [2]:
#make the directory named QC_metrics_output as the current working directory
os.getcwd( )
os.chdir('/home/shayantan/Desktop/GSOC23/GSOC_final/QC_metrics_output/') 

#### FastQC reports are stored in zip files. In the following function, I extract the contents of all 10 fastqc reports and store it in respecteive folders with the same name

In [3]:
def extract_raw_fastqc_data(working_directory):
    for file in os.listdir(working_directory):   # get the list of files
        if zipfile.is_zipfile(file): # if it is a zipfile, extract it
            with zipfile.ZipFile(file) as item: # treat the file as a zip
                item.extractall()  # extract it in the working directory
working_directory = 'fastqc_output/'
extract_raw_fastqc_data(working_directory)

#### FastQC reports contain a text file with all the metrics. In the following function, I extract the contents of the text files of all 10 fastqc reports and store it in seperate lists with the indicative names

In [4]:
rootdir = "fastqc_output/"
def extract_text_data_from_fastqc_reports(rootdir):
    basic_stats=[]; pbsq =[]; pbqs=[]; pbseqcon=[]; psgc=[]; pbnc=[]; snl=[]; sdl=[]; ac=[]
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            f = FastQCParser(d+'/fastqc_data.txt') 
            basic_stats.append(f['Basic Statistics'])
            pbsq.append(f['Per base sequence quality'])
            pbqs.append(f['Per sequence quality scores'])
            pbseqcon.append(f['Per base sequence content'])
            psgc.append(f['Per sequence GC content'])
            pbnc.append(f['Per base N content'])
            snl.append(f['Sequence Length Distribution'])
            sdl.append(f['Sequence Duplication Levels'])
            ac.append(f['Adapter Content'])
                     
    return basic_stats, pbsq, pbqs, pbseqcon, psgc, pbnc, snl, sdl, ac

#aggregating stats and metrics for all 10 files
basic_stats, pbsq, pbqs, pbseqcon, psgc, pbnc, snl, sdl, ac= extract_text_data_from_fastqc_reports(rootdir)

#### The following function converts the basic stats field in the fastqc text file to a data frame

In [5]:
def basic_stats_to_dataframe(basic_stats):
    colnames = ['Filename', 'File type', 'Encoding', 'Total Sequences',
           'Sequences flagged as poor quality', 'Sequence length', '%GC', 'status']
    df = pd.DataFrame(columns=colnames)
    for i in range(0,len(basic_stats)):
        lst = pd.DataFrame(basic_stats[i]['data'])
        lst = lst.transpose().iloc[1,].to_list()
        lst.append(basic_stats[0]['status'])
        df.loc[i] = lst
        
    return df

#converting basic stats data dump into meaningful rows and columns
df_basic_stats=basic_stats_to_dataframe(basic_stats)

In [6]:
df_basic_stats

,Filename,File type,Encoding,Total Sequences,Sequences flagged as poor quality,Sequence length,%GC,status
0,SRR925780_1.bam,Conventional base calls,Sanger / Illumina 1.9,12734154,0,90-108,51,pass
1,SRR925780_10.bam,Conventional base calls,Sanger / Illumina 1.9,3658106,0,90-108,49,pass
2,SRR925780_5.bam,Conventional base calls,Sanger / Illumina 1.9,3976243,0,90-108,49,pass
3,SRR925780_7.bam,Conventional base calls,Sanger / Illumina 1.9,4950428,0,90-108,50,pass
4,SRR925780_6.bam,Conventional base calls,Sanger / Illumina 1.9,3193258,0,90-108,49,pass
5,SRR925780_9.bam,Conventional base calls,Sanger / Illumina 1.9,4801748,0,90-108,50,pass
6,SRR925780_2.bam,Conventional base calls,Sanger / Illumina 1.9,4866379,0,90-108,49,pass
7,SRR925780_4.bam,Conventional base calls,Sanger / Illumina 1.9,2215355,0,90-108,47,pass
8,SRR925780_8.bam,Conventional base calls,Sanger / Illumina 1.9,2266667,0,90-108,49,pass
9,SRR925780_3.bam,Conventional base calls,Sanger / Illumina 1.9,5087488,0,90-108,49,pass


#### The following function converts the per base sequence quality field in the fastqc text file to a data frame

In [7]:
def per_base_sequence_quality_to_dataframe(pbsq):
    colnames = ['Base',
   'Mean',
   'Median',
   'Lower Quartile',
   'Upper Quartile',
   '10th Percentile',
   '90th Percentile','Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(pbsq)):
        lst = pd.DataFrame(pbsq[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df

#converting basic stats data dump into meaningful rows and columns
df_pbsq=per_base_sequence_quality_to_dataframe(pbsq)

In [8]:
df_pbsq

,Base,Mean,Median,Lower Quartile,Upper Quartile,10th Percentile,90th Percentile,Filename
0,1,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_1.bam
1,2,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_1.bam
2,3,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_1.bam
3,4,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_1.bam
4,5,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_1.bam
...,...,...,...,...,...,...,...,...
54,100-101,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_3.bam
55,102-103,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_3.bam
56,104-105,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_3.bam
57,106-107,30.0,30.0,30.0,30.0,30.0,30.0,SRR925780_3.bam


#### The following function converts the per sequence quality scores field in the fastqc text file to a data frame

In [9]:
def per_sequence_quality_scores_to_dataframe(pbqs):
    colnames = ['Quality', 'Count', 'status','Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(pbqs)):
        lst = pd.DataFrame(pbqs[i]['data'])
        lst['status'] = pbqs[i]['status']
        lst['Filename'] = df_basic_stats['Filename'][i]
        l.append(lst)
    
    df = pd.concat(l)
    df.columns = colnames

    return df

#converting basic stats data dump into meaningful rows and columns
df_pbqs=per_sequence_quality_scores_to_dataframe(pbqs)

In [10]:
df_pbqs

,Quality,Count,status,Filename
0,30,12734154.0,pass,SRR925780_1.bam
0,30,3658106.0,pass,SRR925780_10.bam
0,30,3976243.0,pass,SRR925780_5.bam
0,30,4950428.0,pass,SRR925780_7.bam
0,30,3193258.0,pass,SRR925780_6.bam
0,30,4801748.0,pass,SRR925780_9.bam
0,30,4866379.0,pass,SRR925780_2.bam
0,30,2215355.0,pass,SRR925780_4.bam
0,30,2266667.0,pass,SRR925780_8.bam
0,30,5087488.0,pass,SRR925780_3.bam


#### The following function converts the per base sequence content field in the fastqc text file to a data frame

In [11]:
def per_base_sequence_content_to_dataframe(pbseqcon):
    colnames = ['Base', 'G', 'A', 'T', 'C','Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(pbseqcon)):
        lst = pd.DataFrame(pbseqcon[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df
#converting basic stats data dump into meaningful rows and columns
df_pbseqcon=per_base_sequence_content_to_dataframe(pbseqcon)

In [12]:
df_pbseqcon

,Base,G,A,T,C,Filename
0,1,22.714901,26.396870,25.291390,25.596839,SRR925780_1.bam
1,2,26.676164,27.106223,23.353423,22.864190,SRR925780_1.bam
2,3,25.979282,24.392367,24.178939,25.449412,SRR925780_1.bam
3,4,25.493076,24.222081,24.248427,26.036416,SRR925780_1.bam
4,5,25.218205,24.340092,25.204895,25.236809,SRR925780_1.bam
...,...,...,...,...,...,...
54,100-101,25.585431,26.278225,24.184014,23.952331,SRR925780_3.bam
55,102-103,25.927256,26.711415,23.844966,23.516364,SRR925780_3.bam
56,104-105,26.164777,26.842385,23.702121,23.290718,SRR925780_3.bam
57,106-107,26.931082,26.799761,23.169390,23.099768,SRR925780_3.bam


#### The following function converts the per sequence GC content field in the fastqc text file to a data frame

In [13]:
def per_sequence_GC_content_to_dataframe(psgc):
    colnames = ['GC Content', 'Count', 'Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(psgc)):
        lst = pd.DataFrame(psgc[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df
#converting basic stats data dump into meaningful rows and columns
df_psgc=per_sequence_GC_content_to_dataframe(psgc)

In [14]:
df_psgc

,GC Content,Count,Filename
0,0,62.0,SRR925780_1.bam
1,1,45.0,SRR925780_1.bam
2,2,24.0,SRR925780_1.bam
3,3,20.5,SRR925780_1.bam
4,4,22.5,SRR925780_1.bam
...,...,...,...
96,96,6.0,SRR925780_3.bam
97,97,2.5,SRR925780_3.bam
98,98,1.5,SRR925780_3.bam
99,99,1.0,SRR925780_3.bam


#### The following function converts the per base N content field in the fastqc text file to a data frame

In [15]:
def per_base_N_content(pbnc):
    colnames = ['Base', 'N-Count', 'Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(pbnc)):
        lst = pd.DataFrame(pbnc[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df
#converting basic stats data dump into meaningful rows and columns
df_pbnc=per_base_N_content(pbnc)

In [16]:
df_pbnc

,Base,N-Count,Filename
0,1,0.077767,SRR925780_1.bam
1,2,0.004264,SRR925780_1.bam
2,3,0.000597,SRR925780_1.bam
3,4,0.000974,SRR925780_1.bam
4,5,0.000314,SRR925780_1.bam
...,...,...,...
54,100-101,0.851343,SRR925780_3.bam
55,102-103,0.874293,SRR925780_3.bam
56,104-105,0.777135,SRR925780_3.bam
57,106-107,0.684259,SRR925780_3.bam


#### The following function converts the sequence length distribution field in the fastqc text file to a data frame

In [17]:
def sequence_length_dist(snl):
    colnames = ['Length', 'Count', 'Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(snl)):
        lst = pd.DataFrame(snl[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df
#converting basic stats data dump into meaningful rows and columns
df_snl=per_base_N_content(snl)

In [18]:
df_snl

,Base,N-Count,Filename
0,90,6715387.0,SRR925780_1.bam
1,91,0.0,SRR925780_1.bam
2,92,0.0,SRR925780_1.bam
3,93,0.0,SRR925780_1.bam
4,94,0.0,SRR925780_1.bam
...,...,...,...
14,104,0.0,SRR925780_3.bam
15,105,0.0,SRR925780_3.bam
16,106,0.0,SRR925780_3.bam
17,107,0.0,SRR925780_3.bam


#### The following function converts the sequence duplication levels field in the fastqc text file to a data frame

In [19]:
def sequence_duplication_levels(sdl):
    colnames = ['Duplication Level',
  'Percentage of deduplicated',
  'Percentage of total', 'Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(sdl)):
        lst = pd.DataFrame(sdl[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df
#converting basic stats data dump into meaningful rows and columns
df_sdl=sequence_duplication_levels(sdl)

In [20]:
df_sdl

,Duplication Level,Percentage of deduplicated,Percentage of total,Filename
0,1,82.473582,63.699434,SRR925780_1.bam
1,2,11.401289,17.611837,SRR925780_1.bam
2,3,3.414397,7.911449,SRR925780_1.bam
3,4,1.353629,4.181964,SRR925780_1.bam
4,5,0.622107,2.402457,SRR925780_1.bam
...,...,...,...,...
11,>100,0.000000,0.000000,SRR925780_3.bam
12,>500,0.000000,0.000000,SRR925780_3.bam
13,>1k,0.000000,0.000000,SRR925780_3.bam
14,>5k,0.000000,0.000000,SRR925780_3.bam


#### The following function converts the adapter content field in the fastqc text file to a data frame

In [21]:
def adapter_content(ac):
    colnames = ['Position',
  'Illumina Universal Adapter',
  "Illumina Small RNA 3' Adapter",
  "Illumina Small RNA 5' Adapter",
  'Nextera Transposase Sequence',
  'SOLID Small RNA Adapter', 'Filename']
    df = pd.DataFrame(columns=colnames)
    l=[]
    for i in range(0,len(ac)):
        lst = pd.DataFrame(ac[i]['data'])
        lst['Filename'] = np.repeat(np.array(df_basic_stats['Filename'][i]), lst.shape[0])
        l.append(lst)
        
    df = pd.concat(l)
    df.columns = colnames

    return df
#converting basic stats data dump into meaningful rows and columns
df_ac=adapter_content(ac)

In [22]:
df_ac

,Position,Illumina Universal Adapter,Illumina Small RNA 3' Adapter,Illumina Small RNA 5' Adapter,Nextera Transposase Sequence,SOLID Small RNA Adapter,Filename
0,1,0.000000,0.000008,0.0,0.000000,0.000000,SRR925780_1.bam
1,2,0.000000,0.000008,0.0,0.000000,0.000016,SRR925780_1.bam
2,3,0.000000,0.000008,0.0,0.000000,0.000024,SRR925780_1.bam
3,4,0.000000,0.000016,0.0,0.000000,0.000024,SRR925780_1.bam
4,5,0.000000,0.000016,0.0,0.000000,0.000024,SRR925780_1.bam
...,...,...,...,...,...,...,...
48,88-89,1.017329,0.000020,0.0,0.001297,0.000079,SRR925780_3.bam
49,90-91,1.065496,0.000020,0.0,0.001297,0.000079,SRR925780_3.bam
50,92-93,1.115895,0.000020,0.0,0.001297,0.000079,SRR925780_3.bam
51,94-95,1.175796,0.000020,0.0,0.001297,0.000079,SRR925780_3.bam


#### The following function deals with the output from Picard tool. It reads all 10 tsv files from the 'cleaned' directory and stores the dataframes into a list

In [47]:
rootdir = 'picard_output/cleaned'
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    df_picard.append(pd.read_csv(d, sep="\t"))

In [50]:
df_picard[0].head(3)

,CATEGORY,FIRST_OF_PAIR,SECOND_OF_PAIR,PAIR,UNPAIRED,READ_LENGTH,20,21,22,23,...,99,100,101,102,103,104,105,106,107,108
0,TOTAL_READS,2462107.0,2462025.0,4924132.0,163356.0,PAIRED_TOTAL_LENGTH_COUNT,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2280641.0
1,PF_READS,2462107.0,2462025.0,4924132.0,163356.0,PAIRED_ALIGNED_LENGTH_COUNT,2288.0,2185.0,1992.0,1904.0,...,3859.0,6198.0,9764.0,2896.0,3880.0,6096.0,9629.0,13199.0,4813.0,2080113.0
2,PCT_PF_READS,1.0,1.0,1.0,1.0,UNPAIRED_TOTAL_LENGTH_COUNT,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,89732.0
